<a href="https://colab.research.google.com/github/adeokar/dq/blob/master/Copy_of_Data_Cleaning_and_Preparation_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7: Data Cleaning and Preparation

Author: Amit Deokar, Ph.D.

Based on: \
*Chapter 7: Data Cleaning and Preparation*, **Python for Data Analysis (2nd Edition)** by **Wes McKinney**

Importing libraries:

In [0]:
import numpy as np
np.random.seed(12345)
np.set_printoptions(precision=4, suppress=True)

import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

In [0]:
pd.__version__

In [0]:
!pip install -U pandas

## 7.1 Handling Missing Data

**Pandas Documentation for [Working with Missing Data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)**

For numeric data, pandas uses the floating-point value `NaN` (Not a Number) to represent missing data. It is called a **sentinel value** that can be easily detected:

In [0]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [0]:
string_data

In [0]:
string_data.isnull()

- Pandas adopts a convention used in the R programming language by referring to missing data as **NA**, which stands for *not available*. 
- In statistics applications, NA data may either be data that *does not exist* or that *exists but was not observed* (through problems with data collection, for example). 
- When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

#### NA handling methods
- `dropna()`: Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.
- `fillna()`: Fill in missing data with some value or using an interpolation method such as `ffill` or `bfill`.
- `isnull()`: Return boolean values indicating which values are missing/NA.
- `notnull()`: Negation of `isnull`.

#### None (Pythonic missing data) vs NaN (Missing numerical data)
The built-in Python `None` value is also treated as NA in object arrays. 
- Since `None` is a Python object, it cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type `'object'`. (i.e., arrays of Python objects).
- `dtype = object` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects.
- While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types.

Reference: [Handling Missing Data](https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html) by Jake VanderPlas

In [0]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

- The use of Python objects in an array also means that if you perform aggregations like `sum()` or `min()` across an array with a `None` value, you will generally get an error:

In [0]:
vals1 = np.array([1, None, 3, 4])
vals1

In [0]:
#vals1.sum() # gives error

This reflects the fact that addition between an integer and `None` is undefined.

However, object arrays with a `None` value can be used to check for missing values using methods like `isnull()` and `notnull()`.

In [0]:
string_data.dtype

In [0]:
string_data[0] = None

In [0]:
string_data.isnull()

#### NaN and None in Pandas

Reference: [Handling Missing Data](https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html) by Jake VanderPlas

`NaN` and `None` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [0]:
pd.Series([1, np.nan, 2, None])

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. For example, if we set a value in an integer array to `np.nan`, it will automatically be upcast to a floating-point type to accommodate the NA:

In [0]:
x = pd.Series(range(2), dtype=int)
x

In [0]:
x[0] = None
x

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the `None` to a `NaN` value.

There is an experimental feature in pandas version 1.0.1 as described in the **Pandas Documentation for [Working with Missing Data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)**

> - Starting from pandas 1.0, an experimental `pd.NA` value (singleton) is available to represent scalar missing values. At this moment, it is used in the nullable integer, boolean and dedicated string data types as the missing value indicator.
> - The goal of `pd.NA` is provide a “missing” indicator that can be used consistently across data types (instead of `np.nan`, `None` or `pd.NaT` depending on the data type).

In [0]:
s = pd.Series([1, 2, None], dtype="Int64")
s

From the **Pandas Documentation for [Working with Missing Data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)**: 

> Currently, pandas does not yet use those data types by default (when creating a DataFrame or Series, or when reading in data), so you need to specify the dtype explicitly. An easy way to convert to those dtypes is explained [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#missing-data-na-conversion).

> - If you have a DataFrame or Series using traditional types that have missing data represented using `np.nan`, there are convenience methods [`convert_dtypes()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.convert_dtypes.html#pandas.DataFrame.convert_dtypes) in Series and [`convert_dtypes()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.convert_dtypes.html#pandas.DataFrame.convert_dtypes) in DataFrame that can convert data to use the newer dtypes for integers, strings and booleans listed [here](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html#basics-dtypes). 
> - This is especially helpful after reading in data sets when letting the readers such as `read_csv()` and `read_excel()` infer default dtypes.

In [0]:
x.convert_dtypes()

### Filtering Out Missing Data

While you always have the option to do it by hand using `pandas.isnull()` and boolean indexing, `dropna()` can be helpful.

#### For Series:

In [0]:
from numpy import nan as NA

data = pd.Series([1, NA, 3.5, NA, 7]) # alternatively, import numpy as np, and then use, data = pd.Series([1, np.nan, 3.5, np.nan, 7])

In [0]:
data

To filter out rows that contain missing values:

In [0]:
data.dropna()

To filter out rows that contain missing values (alternative way):

In [0]:
data[data.notnull()]

To select rows that contain missing values:

In [0]:
data[data.isnull()]

#### For DataFrames:

In [0]:
from numpy import nan as NA
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

In [0]:
data

`dropna` by default drops any row containing a missing value:

In [0]:
cleaned = data.dropna()

In [0]:
cleaned

Passing `how='all'` will only drop rows that are all NA:

In [0]:
data.dropna(how='all')

To drop columns that are all NA, pass `axis=1` along with `how='all'`:

In [0]:
data[4] = NA

In [0]:
data

In [0]:
data.dropna(axis=1, how='all')

To keep only rows containing a certain number of observations, use the `thresh` (require those many non-NA values) argument:

In [0]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

In [0]:
df.dropna()

In [0]:
df.dropna(thresh=2)

### Filling In Missing Data

- Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways.

In [0]:
df.fillna(0)

- Calling `fillna` with a dict, you can use a different fill value for each column:

In [0]:
df.fillna({1: 0.5, 2: 0})

- `fillna` returns a new object, but you can modify the existing object in-place:

In [0]:
_ = df.fillna(0, inplace=True)
df

The same interpolation methods available for reindexing can be used with `fillna`.

#### `fillna` function arguments
- `value`: Scalar value or dict-like object to use to fill missing values
- `method`: Interpolation; by default `'ffill'` if function called with no other arguments
- `axis`: Axis to fill on; default `axis=0`
- `inplace`: Modify the calling object without producing a copy
- `limit`: For forward and backward filling, maximum number of consecutive periods to fill

In [0]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

In [0]:
df.fillna(method='ffill')

In [0]:
df.fillna(method='ffill', limit=2)

With `fillna` you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:

In [0]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

In [0]:
data.fillna(data.mean())

## 7.2 Data Transformation

### Removing Duplicates

In [0]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [0]:
data.duplicated() # checks if each row is a duplicate (has been observed in a previous row)

In [0]:
data.drop_duplicates()

Both of these methods by default consider all of the columns.

In [0]:
data['v1'] = range(7)
data

- You can specify any subset of columns to detect duplicates.

In [0]:
data.drop_duplicates(['k1']) # filter duplicates only based on 'k1' column

- Passing `keep='last'` will return the last duplicate row.

In [0]:
data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming Data Using a Function or Mapping

In [0]:
sales = {
    "month": ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
    "candy" : [134, 65, 89 ,24, 45, 23, 27, 88, 45, 90, 101, 120],
    "sugar" : [110.0, 45.0, None, 77.0, None, 50.0, 77.0, None, 80.0, 11.0, 90.0, 115.0],
    "cookies" : [119, 78, 56, 45, 33, 91, 67, 47, 19, 97, 111, 129]
}

data = pd.DataFrame(sales)
data

In [0]:
month_to_emp = {
    'jan': 'Anita',
    'feb': 'Raghu',
    'mar': 'Kavita',
    'apr': 'Matt',
    'may': 'Amy',
    'jun': 'Anita',
    'jul': 'Raghu',
    'aug': 'Anita',
    'sep': 'Kavita',
    'oct': 'Martha',
    'nov': 'Amy',
    'dec': 'Raghu'
}

In [0]:
lowercased = data['month'].str.lower()
lowercased

- Using `map` is a convenient way to perform element-wise transformations and other data cleaning–related operations.
- The `map` method on a Series accepts a function or dict-like object containing a mapping.

In [0]:
data['emp'] = lowercased.map(month_to_emp)
data

In [0]:
data['month'].map(lambda x: month_to_emp[x.lower()])
data

### Replacing Values

- Filling in missing data with the `fillna` method is a special case of more general value replacement. 
- As you’ve already seen, map can be used to modify a subset of values in an object but `replace` provides a simpler and more flexible way to do so.
- `replace` produces a new Series (unless you pass `inplace=True`).

In [0]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [0]:
data.replace(-999, np.nan) # The -999 values might be sentinel values for missing data.

To replace multiple values at once, you instead pass a list and then the substitute value:

In [0]:
data.replace([-999, -1000], np.nan)

To use a different replacement for each value, pass a list of substitutes:

In [0]:
data.replace([-999, -1000], [np.nan, 0])

The argument passed can also be a dict:

In [0]:
data.replace({-999: np.nan, -1000: 0})

### Renaming Axis Indexes

- Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects.
- Again, use `inplace=True` for modifying in-place without creating a new data structure.

In [0]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [0]:
data.index

In [0]:
transform = lambda x: x[:4].upper()

In [0]:
data.index.map(transform)

- You can assign to index, modifying the DataFrame in-place:

In [0]:
data.index = data.index.map(transform)
data

To create a transformed version of a dataset without modifying the original, a useful method is `rename`:

In [0]:
data.rename(index=str.title, columns=str.upper)

In [0]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

- `rename` saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes. 
- Should you wish to modify a dataset in-place, pass `inplace=True`.

In [0]:
data

In [0]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and Binning
- Continuous data is often discretized or otherwise separated into “bins” for analysis.

In [0]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [0]:
bins = [18, 25, 35, 60, 100]

In [0]:
cats = pd.cut(ages, bins)
cats

- The object pandas returns is a special `Categorical` object.
- You can treat it like an array of strings indicating the bin name; internally it contains a `categories` array specifying the distinct category names along with a labeling for the `ages` data in the codes attribute.

In [0]:
cats.codes

In [0]:
cats.categories

- `pd.value_counts()` gives bin counts for the result of `pandas.cut`

In [0]:
pd.value_counts(cats)

In [0]:
pd.value_counts(pd.cut(ages, bins))

- Consistent with mathematical notation for intervals, a parenthesis means that the side is _open_, while the square bracket means it is _closed_ (inclusive). 
- You can change which side is closed by passing `right=False`.

In [0]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

- You can also pass your own bin names by passing a list or array to the `labels` option:

In [0]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

- If you pass an integer number of bins to `cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.

In [0]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2) # precision=2 option limits the decimal precision to two digits.

- `qcut` function bins the data based on sample quantiles.
- Depending on the distribution of the data, using `cut` will not usually result in each bin having the same number of data points.
- Since `qcut` uses sample quantiles instead, by definition you will obtain roughly equal-size bins.

In [0]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4, precision=2)  # Cut into quartiles
cats

In [0]:
pd.value_counts(cats)

In [0]:
pd.value_counts(pd.cut(data, 4, precision=2)) # compare cut to qcut

- Similar to `cut` you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [0]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.], precision=2)

In [0]:
pd.value_counts(pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.], precision=2))

### Detecting and Filtering Outliers
- Filtering or transforming outliers is largely a matter of applying array operations.

In [0]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.head()

In [0]:
data.describe()   # use data.describe().round(x) for rounding off each value in the dataframe to x decimal places.

Demo: Suppose you wanted to find values in one of the columns exceeding 3 in absolute value.

In [0]:
col = data[2]

In [0]:
col[np.abs(col) > 3]

To select all rows having a value exceeding 3 or –3, you can use the [`any`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.any.html#pandas.DataFrame.any) method on a boolean DataFrame:

In [0]:
data[(np.abs(data) > 3).any(axis=1)]

Values can be set based on these criteria. For example, to cap values outside the interval –3 to 3:

In [0]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [0]:
data.describe()

- The statement `np.sign(data)` produces 1 and –1 values based on whether the values in data are positive or negative. **Documentation link for [`np.sign()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sign.html)**

In [0]:
np.sign(data).head()

### Permutation and Random Sampling

- Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the `numpy.random.permutation` function. **Documentation link for [`np.random.permutation()`](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.permutation.html)**
- Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering.

In [0]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

In [0]:
sampler = np.random.permutation(5)
sampler

In [0]:
df

- **Documentation link for [`df.take()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.take.html)**

In [0]:
df.take(sampler)

- To select a random subset without replacement, you can use the `sample` method on Series and DataFrame. 
- **Documentation link for [`df.sample()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html)**

In [0]:
df.sample(n=3)

- To generate a sample *with* replacement (to allow repeat choices), pass `replace=True` to `sample()`.

In [0]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

### Computing Indicator/Dummy Variables

- Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a “dummy” or “indicator” matrix. 
- If a column in a DataFrame has `k` distinct values, you would derive a matrix or DataFrame with `k` columns containing all 1s and 0s. 
- pandas has a `get_dummies` function for doing this, though devising one yourself is not difficult. 
- **Documentation link for [`pd.get_dummies()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)**

In [0]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

- To add a prefix to the columns in the indicator DataFrame, pass the `prefix` argument to the `get_dummies()` function.

In [0]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

In [0]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

To create dummies if a row in a DataFrame belongs to multiple categories:

In [0]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('pydata-book-2nd-edition/datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames, engine='python')
type(movies)

In [0]:
movies[:10]

Adding indicator variables for each genre requires a little bit of wrangling. 

First, we extract the list of unique genres in the dataset:

In [0]:
all_genres = []

for x in movies.genres:
    all_genres.extend(x.split('|'))

In [0]:
genres = pd.unique(all_genres)
genres

Next, start with a DataFrame of all zeros

In [0]:
len(movies)

In [0]:
len(genres)

In [0]:
movies.shape

In [0]:
movies.shape[0]

In [0]:
zero_matrix = np.zeros((len(movies), len(genres)))
zero_matrix

In [0]:
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

Iterate through each movie and set entries in each row of dummies to 1. Use the `dummies.columns` to compute the column indices for each genre.

In [0]:
gen = movies.genres[1]
gen

In [0]:
gen.split('|')

- **Documentation link for [`pd.Index.get_indexer()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.get_indexer.html)**

In [0]:
type(dummies.columns)

In [0]:
dummies.columns.get_indexer(gen.split('|'))

In [0]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [0]:
dummies.head()

In [0]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [0]:
movies_windic.iloc[0:5]

Typical recipe for statistical applications is to combine `get_dummies` with a discretization function like `cut`.

In [0]:
np.random.seed(12345)
values = np.random.rand(10)
values

In [0]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
bins

In [0]:
pd.cut(values, bins)

In [0]:
pd.get_dummies(pd.cut(values, bins))

## 7.3 String Manipulation

### String Object Methods

- In many string munging and scripting applications, built-in string methods are sufficient. See **documentation for [Python string methods](https://docs.python.org/3/library/stdtypes.html#string-methods)**.

In [0]:
val = 'a,b,  guido'
val.split(',')

In [0]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [0]:
first, second, third = pieces
first + '::' + second + '::' + third

A faster and more Pythonic way is to pass a list or tuple to the join method on the string '::'.

In [0]:
'::'.join(pieces)

- Other methods are concerned with locating substrings. 
- Using Python’s `in` keyword is the best way to detect a substring, though `index` and `find` can also be used.

In [0]:
'guido' in val

In [0]:
val.index(',')

In [0]:
val.find(':')

_Note the difference between `find` and `index` is that `index` raises an exception if the string isn’t found (versus returning –1)_

In [0]:
#val.index(':')

Relatedly, `count` returns the number of occurrences of a particular substring:

In [0]:
val.count(',')

`replace` is used to substitute occurrences of one pattern for another.

In [0]:
val.replace(',', '::')

`replace` is commonly used to delete patterns, too, by passing an empty string:

In [0]:
val.replace(',', '')

### Regular Expressions

In [0]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

In [0]:
regex = re.compile('\s+')
regex.split(text)

In [0]:
regex.findall(text)

In [0]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [0]:
regex.findall(text)

In [0]:
m = regex.search(text)
m
text[m.start():m.end()]

In [0]:
print(regex.match(text))

In [0]:
print(regex.sub('REDACTED', text))

In [0]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [0]:
m = regex.match('wesm@bright.net')
m.groups()

In [0]:
regex.findall(text)

In [0]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

### Vectorized String Functions in pandas

In [0]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

In [0]:
data.str.contains('gmail')

In [0]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

In [0]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [0]:
matches.str.get(1)
matches.str[0]

In [0]:
data.str[:5]

In [0]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion